# 样例介绍
* 目前工业界通用的人像分割主要采用绿屏技术，需要专门的绿屏设备及环境，不利于普通用户的广泛使用。在这个样例中，我们使用了一个深度学习神经网络PortraitNet，能够实时地进行人像分割和背景替换。

# 前期准备
* 基础镜像的样例目录中已包含转换后的om模型以及测试图片，如果直接运行，可跳过此步骤。如果需要重新转换模型，可参考如下步骤：
* 首先我们可以在[这个链接](https://ascend-repo.obs.cn-east-2.myhuaweicloud.com/Atlas%20200I%20DK%20A2/DevKit/downloads/23.0.RC1/Ascend-devkit_23.0.RC1_downloads.xlsx)的表格中找到本样例的依赖文件，下载我们已经准备好了的TensorFlow模型文件，TensorFlow是开源的深度学习框架。

* 为了能进一步优化模型推理性能，我们需要将其转换为om模型进行使用，以下为转换指令： 
    ```shell
    atc --model=./portrait.pb  --insert_op_conf=./insert_op.cfg  --output="./portrait" --output_type=FP32 --input_shape="Inputs/x_input:1,224,224,3" --framework=3 --soc_version=Ascend310B1
    ```
    其中转换参数的含义为：
    * --model：输入模型路径
    * --insert_op_conf：插入算子的配置文件路径与文件名
    * --output：输出模型路径
    * --output_type：指定网络输出数据类型或指定某个输出节点的输出类型
    * --input_shape：指定模型输入数据的shape
    * --framework：原始网络模型框架类型，3表示TensorFlow
    * --soc_version：昇腾AI处理器型号
        

# 模型推理实现
* 导入代码依赖

In [ ]:
import sys
import os

import numpy as np
import cv2
import matplotlib.pyplot as plt

import acl
import acllite_utils as utils
import constants as const
from acllite_imageproc import AclLiteImageProc
from acllite_model import AclLiteModel
from acllite_image import AclLiteImage
from acllite_resource import resource_list

* 主要推理代码

In [ ]:
class AclLiteResource:
    """AclLite资源管理"""
    def __init__(self, device_id=0):
        self.device_id = device_id
        self.context = None
        self.stream = None
        self.run_mode = None
        
    def init(self):
        """初始化ACL相关资源"""
        print("init resource stage:")
        ret = acl.init()

        ret = acl.rt.set_device(self.device_id)
        utils.check_ret("acl.rt.set_device", ret)

        self.context, ret = acl.rt.create_context(self.device_id)
        utils.check_ret("acl.rt.create_context", ret)

        self.stream, ret = acl.rt.create_stream()
        utils.check_ret("acl.rt.create_stream", ret)

        self.run_mode, ret = acl.rt.get_run_mode()
        utils.check_ret("acl.rt.get_run_mode", ret)

        print("Init resource success")

    def __del__(self):
        """释放ACL相关资源"""
        print("acl resource release all resource")
        resource_list.destroy()
        if self.stream:
            print("acl resource release stream")
            acl.rt.destroy_stream(self.stream)

        if self.context:
            print("acl resource release context")
            acl.rt.destroy_context(self.context)

        print("Reset acl device ", self.device_id)
        acl.rt.reset_device(self.device_id)
        print("Release acl resource success")

In [ ]:
class Seg(object):
    """人像分割模型推理"""
    def __init__(self, model_path, model_width, model_height):
        self._model_path = model_path
        self._model_width = model_width
        self._model_height = model_height
        self.device_id = 0
        self._dvpp = None
        self._model = None

    def init(self):
        """初始化相关资源"""
        # Initialize dvpp
        self._dvpp = AclLiteImageProc()

        # Load model
        self._model = AclLiteModel(self._model_path)

        return const.SUCCESS

    @utils.display_time
    def pre_process(self, image):
        """图片预处理"""
        image_dvpp = image.copy_to_dvpp()
        yuv_image = self._dvpp.jpegd(image_dvpp)
        resized_image = self._dvpp.resize(yuv_image,
                                          self._model_width, self._model_height)
        return resized_image     

    @utils.display_time
    def inference(self, input_data):
        """模型推理"""
        return self._model.execute(input_data)

    @utils.display_time
    def post_process(self, infer_output, image_name):
        """获取分割结果"""
        data = infer_output[0]
        vals = data.flatten()
        mask = np.clip((vals * 255), 0, 255)
        mask = mask.reshape(224, 224, 2)
        cv2.imwrite(os.path.join(MASK_DIR, image_name), mask[:, :, 0])
        return mask 

    
@utils.display_time
def background_replace(bg_path, ori_path, mask_path):
    """将人像分割结果与背景图片结合"""
    background = cv2.imread(bg_path)
    height, width = background.shape[:2]
    ori_img = cv2.imread(ori_path)
    mask = cv2.imread(mask_path, 0)
    mask = mask / 255
    mask_resize = cv2.resize(mask, (width, height))
    ori_img = cv2.resize(ori_img, (width, height))
    mask_bg = np.repeat(mask_resize[..., np.newaxis], 3, 2)
    result = np.uint8(background * mask_bg + ori_img * (1 - mask_bg))
    cv2.imwrite(os.path.join(OUTPUT_DIR, os.path.basename(mask_path)), result)

    
def main():
    """推理主函数"""
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    os.makedirs(MASK_DIR, exist_ok=True)

    acl_resource = AclLiteResource()
    acl_resource.init()

    seg = Seg(MODEL_PATH, MODEL_WIDTH, MODEL_HEIGHT)
    ret = seg.init()
    utils.check_ret("seg.init ", ret)

    images_list = [os.path.join(IMAGE_DIR, img)
                   for img in os.listdir(IMAGE_DIR)
                   if os.path.splitext(img)[1] in const.IMG_EXT]

    for image_file in images_list:
        image_name = os.path.basename(image_file)
        if image_name != 'background.jpg':
            print('====' + image_name + '====')
            # read image
            image = AclLiteImage(image_file)
            # Preprocess the picture
            resized_image = seg.pre_process(image)
            # Inference
            result = seg.inference([resized_image, ])
            # Post-processing
            mask = seg.post_process(result, image_name)
            # Fusion of segmented portrait and background image
            background_replace(os.path.join(IMAGE_DIR, 'background.jpg'), \
                                        image_file, os.path.join(MASK_DIR, image_name))

# 样例运行

* 运行以下代码后，我们就可以看到人像背景替换效果了。左边是原始的人像图片，右边是替换人像背景后的图片。

In [ ]:
currentPath = '.'
OUTPUT_DIR = os.path.join(currentPath, 'out/result')
MASK_DIR = os.path.join(currentPath, 'out/mask')
MODEL_PATH = os.path.join(currentPath,"./model/portrait.om")
IMAGE_DIR = os.path.join(currentPath, "data" )
MODEL_WIDTH = 224
MODEL_HEIGHT = 224

main()

path = os.path.join(IMAGE_DIR, 'ori.jpg')
original = cv2.imread(path)[:, :, ::-1]

path = os.path.join(OUTPUT_DIR, 'ori.jpg')
result = cv2.imread(path)[:, :, ::-1]

fig, axarr = plt.subplots(nrows=1, ncols=2)
for ax, image in zip(axarr, [original, result]):
    ax.imshow(image)
    ax.axis('off')
fig.tight_layout()

# 样例总结与扩展
以上就是这个样例的全部内容了，通过这个样例，我们可以发现借助深度学习神经网络，以往费时费力的人像背景替换工作可以变得如此简单高效。大家可以尝试使用其他图片，测试一下人像背景替换效果哦。